# Premier League Predictor v4.0 (Class Balancing & Model Saving)

### Novidades:
1.  **Forçar Empates:** Vamos aplicar pesos às classes (`sample_weights`) para o modelo deixar de ignorar os empates.
2.  **Instant Load:** O modelo treinado é guardado no disco. Na próxima execução, não precisas de treinar de novo.
3.  **Thresholds Dinâmicos:** Em vez de escolher apenas a maior probabilidade, vamos ver se a probabilidade de empate é "decente" (ex: > 28%) e arriscar.

Imports e Configuração

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib # Para salvar o modelo
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

## 1. Data Acquisition (Recolha de Dados)
Vamos buscar dados reais do `football-data.co.uk`. Vamos carregar várias temporadas consecutivas para que o modelo tenha histórico suficiente para aprender padrões.

* **FTHG**: Full Time Home Goals
* **FTAG**: Full Time Away Goals
* **FTR**: Full Time Result (H=Home, D=Draw, A=Away)

In [ ]:
DATA_FILE = 'premier_league_v3_full.csv'

def get_data(start_year, end_year):
    # 1. Verificar se ficheiro existe localmente
    if os.path.exists(DATA_FILE):
        print(f"Ficheiro local '{DATA_FILE}' encontrado! A carregar...")
        df = pd.read_csv(DATA_FILE)
        df['Date'] = pd.to_datetime(df['Date'])
        # Se quiseres atualizar dados recentes, apaga o ficheiro .csv da pasta e corre isto de novo
        return df
    
    # 2. Se não existe, sacar da net
    print("Ficheiro não encontrado. A fazer download da internet...")
    base_url = "https://www.football-data.co.uk/mmz4281/{}/{}.csv"
    dfs = []
    
    for year in range(start_year, end_year + 1):
        season_str = f"{str(year)[-2:]}{str(year+1)[-2:]}"
        url = base_url.format(season_str, "E0")
        try:
            df = pd.read_csv(url)
            df['Season'] = year
            # Normalizar Data
            df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
            dfs.append(df)
        except Exception as e:
            print(f"Erro no ano {year}: {e}")
            
    full_df = pd.concat(dfs, ignore_index=True)
    full_df = full_df.dropna(subset=['Date', 'FTR'])
    full_df = full_df.sort_values('Date').reset_index(drop=True)
    
    # Guardar para a próxima vez
    full_df.to_csv(DATA_FILE, index=False)
    print("✅ Download concluído e guardado no PC.")
    return full_df

# Carregar dados
df = get_data(2005, 2025)
display(df.tail(3))

## 2. Feature Engineering Completa (ELO + Stats + Odds)

Aqui adicionamos as colunas B365H, B365D, B365A (Odds da Bet365).

In [ ]:
def prepare_features(df, window=5):
    df = df.copy()
    
    # --- 1. ELO SYSTEM ---
    elo_dict = {}
    df['HomeElo'] = 1500.0
    df['AwayElo'] = 1500.0
    k_factor = 20
    
    for i, row in df.iterrows():
        h, a, res = row['HomeTeam'], row['AwayTeam'], row['FTR']
        h_elo = elo_dict.get(h, 1500.0)
        a_elo = elo_dict.get(a, 1500.0)
        
        df.at[i, 'HomeElo'] = h_elo
        df.at[i, 'AwayElo'] = a_elo
        
        if res == 'H': val = 1
        elif res == 'D': val = 0.5
        else: val = 0
        
        exp_h = 1 / (1 + 10**((a_elo - h_elo)/400))
        new_h = h_elo + k_factor * (val - exp_h)
        new_a = a_elo + k_factor * ((1-val) - (1-exp_h))
        
        elo_dict[h] = new_h
        elo_dict[a] = new_a
        
    df['EloDiff'] = df['HomeElo'] - df['AwayElo']
    
    # --- 2. ROLLING STATS ---
    home_stats = df[['Date', 'HomeTeam', 'FTHG', 'FTAG', 'HS', 'HST', 'HC']].copy()
    home_stats.columns = ['Date', 'Team', 'Goals', 'Conceded', 'Shots', 'SoT', 'Corners']
    home_stats['Points'] = df['FTR'].map({'H':3, 'D':1, 'A':0})
    
    away_stats = df[['Date', 'AwayTeam', 'FTAG', 'FTHG', 'AS', 'AST', 'AC']].copy()
    away_stats.columns = ['Date', 'Team', 'Goals', 'Conceded', 'Shots', 'SoT', 'Corners']
    away_stats['Points'] = df['FTR'].map({'A':3, 'D':1, 'H':0})
    
    all_stats = pd.concat([home_stats, away_stats]).sort_values(['Team', 'Date'])
    
    metrics = ['Points', 'Goals', 'Conceded', 'Shots', 'SoT', 'Corners']
    for m in metrics:
        all_stats[f'Avg_{m}'] = all_stats.groupby('Team')[m].transform(
            lambda x: x.shift(1).rolling(window, min_periods=3).mean()
        )
    
    # Merge Home rolling
    df = df.merge(
        all_stats[['Date', 'Team'] + [f'Avg_{m}' for m in metrics]],
        left_on=['Date', 'HomeTeam'],
        right_on=['Date', 'Team'],
        how='left'
    ).drop(columns=['Team'])
    df = df.rename(columns={f'Avg_{m}': f'Home_{m}' for m in metrics})
    
    # Merge Away rolling
    df = df.merge(
        all_stats[['Date', 'Team'] + [f'Avg_{m}' for m in metrics]],
        left_on=['Date', 'AwayTeam'],
        right_on=['Date', 'Team'],
        how='left'
    ).drop(columns=['Team'])
    df = df.rename(columns={f'Avg_{m}': f'Away_{m}' for m in metrics})
    
    # --- 3. BETTING ODDS ---
    if 'B365H' in df.columns:
        df['Prob_Home'] = 1 / df['B365H']
        df['Prob_Draw'] = 1 / df['B365D']
        df['Prob_Away'] = 1 / df['B365A']
        df = df.dropna(subset=['Prob_Home'])
    
    # Preencher só as rolling averages
    rolling_cols = [f for f in df.columns if f.startswith("Home_") or f.startswith("Away_")]
    df[rolling_cols] = df[rolling_cols].fillna(0)
    
    # Remover colunas totalmente vazias
    df = df.dropna(axis=1, how='all')
    
    return df, elo_dict

# Aplicar e verificar colunas
df_processed, elo_tracker = prepare_features(df)
print("Colunas disponíveis para treino:", df_processed.columns.tolist())

## 3. Preparação e Treino do Modelo
Treino Intensivo: Grid Search (Hyperparameter Tuning) Aqui é onde "apertamos" o modelo. Vamos testar várias combinações. Nota: Isto pode demorar 2 ou 3 minutos a correr.

In [ ]:
# 1. Preparar Features
features = ['HomeElo', 'AwayElo', 'EloDiff', 
            'Prob_Home', 'Prob_Draw', 'Prob_Away'] + \
           [c for c in df_processed.columns if 'Home_' in c or 'Away_' in c]
features = [f for f in features if f in df_processed.columns]

# Target
le = LabelEncoder()
df_processed['Target'] = le.fit_transform(df_processed['FTR'])
# 0=Away, 1=Draw, 2=Home (Verifica sempre com le.classes_)

# Split Temporal
split = int(len(df_processed) * 0.90)
train = df_processed.iloc[:split]
test = df_processed.iloc[split:]

X_train = train[features]
y_train = train['Target']
X_test = test[features]
y_test = test['Target']

# --- A MUDANÇA CRUCIAL: Sample Weights ---
# Vamos calcular pesos para equilibrar. 
# Se houver poucos empates, eles ganham um peso gigante.
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

print("⚖️ A treinar com pesos equilibrados (Obrigando a olhar para o Empate)...")

# Modelo (Usamos os melhores params que descobriste ou um set robusto)
model_v4 = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.03,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective='multi:softprob'
)

# Passamos os pesos no .fit()
model_v4.fit(X_train, y_train, sample_weight=sample_weights)

print("✅ Modelo treinado.")

# --- GUARDAR O MODELO (O teu pedido) ---
joblib.dump(model_v4, 'model_xgboost_v4.pkl')
joblib.dump(le, 'label_encoder.pkl') # Importante guardar o descodificador
print("💾 Modelo salvo como 'model_xgboost_v4.pkl'.")

### Matriz de Confusão e accuracy
Vamos ver visualmente onde o modelo erra.
* Eixo Y: O que realmente aconteceu.
* Eixo X: O que o modelo previu.

In [ ]:
preds = model_v4.predict(X_test)
acc = accuracy_score(y_test, preds)

print(f"🎯 Accuracy V4: {acc:.2%}")

# Matriz de Confusão
cm = confusion_matrix(y_test, preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Matriz de Confusão (Com Pesos)')
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.show()

# Classification Report (Para ver a precisão específica dos Empates/Draws)
print(classification_report(y_test, preds, target_names=le.classes_))

## 4. Aplicação na "Vida Real"
Aqui está a função final. Ela usa o dicionário `current_elo` (que contém os valores mais recentes após o último jogo do dataset) para fazer previsões sobre jogos futuros.

In [ ]:
def predict_smart(home, away, odd_h, odd_d, odd_a):
    # Carregar modelo se não estiver em memória
    if 'model_v4' not in locals():
        model_v4 = joblib.load('model_xgboost_v4.pkl')
        print("📂 Modelo carregado do disco.")

    # 1. Buscar dados (Igual ao v3)
    # (Simplificação: assume que elo_tracker e df_processed estão em memória)
    h_elo = elo_tracker.get(home, 1500)
    a_elo = elo_tracker.get(away, 1500)
    
    input_data = {
        'HomeElo': h_elo, 'AwayElo': a_elo, 'EloDiff': h_elo - a_elo,
        'Prob_Home': 1/odd_h, 'Prob_Draw': 1/odd_d, 'Prob_Away': 1/odd_a
    }
    
    # Preencher stats históricas
    h_row = df_processed[df_processed['HomeTeam'] == home].iloc[-1]
    a_row = df_processed[df_processed['AwayTeam'] == away].iloc[-1]
    
    for feat in features:
        if feat not in input_data:
            if 'Home_' in feat: input_data[feat] = h_row[feat]
            elif 'Away_' in feat: input_data[feat] = a_row[feat]

    X_input = pd.DataFrame([input_data])[features]
    
    # 2. Obter Probabilidades Reais
    probs = model_v4.predict_proba(X_input)[0]
    p_away, p_draw, p_home = probs[0], probs[1], probs[2]
    
    print(f"\n🧠 Análise: {home} vs {away}")
    print(f"   Probabilidades: Casa {p_home:.0%} | Empate {p_draw:.0%} | Fora {p_away:.0%}")
    
    # 3. Lógica de Decisão Personalizada (Custom Threshold)
    # Se o Empate for > 30%, vamos considerar muito provável, mesmo que não seja o maior
    # Ajusta este valor (0.30) conforme o teu gosto de risco
    
    prediction = "Inconclusivo"
    
    if p_home > 0.45:
        prediction = f"Vitória {home}"
    elif p_away > 0.45:
        prediction = f"Vitória {away}"
    elif p_draw > 0.28: # Threshold agressivo para empates
        prediction = "EMPATE (Risco calculado)"
    else:
        # Se tudo for baixo (ex: 34, 33, 33), vai pela maior
        max_idx = np.argmax(probs)
        prediction = f"Tendência para {le.classes_[max_idx]}"

    print(f"   >> Veredicto IA: {prediction}")

# Testa com jogos difíceis (Derbies costumam dar empate)
predict_smart('Aston Villa', 'Arsenal', 4.05, 3.45, 1.84)